# Naive Bayes

In deze opdracht ga je Naive Bayes gebruiken om het onderscheid tussen berichten uit twee verschillende nieuwsgroepen te leren.  
Voor informatie over de dataset, zie: http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html

In [2]:
# misc data processing imports
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# dataset
from sklearn.datasets import fetch_20newsgroups

# classifier & testing
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score

# to parse regular expressions
import re

## 1. Data laden

  * Laad de train en test data van de 20newsgroups dataset. 
    * Vraag hierbij alleen de categorieën 'comp.sys.mac.hardware' en 'comp.sys.ibm.pc.hardware' op.
    * Zorg er ook voor dat header, footers en quotes weggelaten worden.
  * Hoeveel train samples (nieuwsgroep artikelen) zijn er? En hoeveel test samples?

In [17]:
categories = ['comp.sys.mac.hardware' ,'comp.sys.ibm.pc.hardware' ]
datasetTrain = fetch_20newsgroups(subset='train', categories=categories,remove=('headers', 'footers', 'quotes','description'),shuffle=True, random_state=42)
datasetTest = fetch_20newsgroups(subset='test', categories=categories,remove=('headers', 'footers', 'quotes','description'),shuffle=True, random_state=42)
print(len(datasetTrain.data))
print(len(datasetTest.data))

777

## 2. Count vectorizer

Je gaat nu zelf de data omzetten naar count vectors, waarop Naive Bayes getraind kan worden.

### 2.1 Pre-processing

  * Splits elk newsgroup artikel in de train dataset in een lijst van afzonderlijke woorden.  
  Vervang daarbij ook alle hoofdletters door kleine letters.
  * Doe bovenstaande ook voor de test dataset.

In [26]:
dsTrainlower= [ re.split('\W+',lines.lower()) for lines in datasetTrain.data] 
dsTestlower= [ re.split('\W+',lines.lower()) for lines in datasetTest.data] 

  * Bouw een vocabulaire op van alle unieke woorden die in de complete **train** dataset voorkomen.

In [33]:
vocabTrain=([word for sublist in dsTrainlower for word in sublist])

In [45]:
vocabTrain = list(set(vocabTrain))

  * Tel voor elk nieuwgroep artikel hoevaak elk woord uit de vocabulaire voorkomt.  
  Het resultaat is een matrix met shape = [n\_train\_samples, n\_words\_in\_vocabulary].

In [62]:
WordFreqTR=np.zeros((len(datasetTrain.data),len(vocabTrain)))

for idx1, trwords in enumerate(dsTrainlower):
    for idx2, vword in enumerate(vocabTrain):
        WordFreqTR[idx1,idx2]=trwords.count(vword)
print(WordFreqTR)


            

[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 2.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]


In [54]:
Matrix = np.zeros((len(datasetTrain.data),len(vocabTrain)))
Matrix.shape

for item in dsTrainlower:
   itemindex = dsTrainlower.index(item)
   for word in item:
       wordindex = vocabTrain.index(word)
       Matrix[itemindex, wordindex] = Matrix[itemindex, wordindex] + 1


In [55]:
print(Matrix)

[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 2.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]


### 2.2 Naive Bayes trainen & testen

  * Maak een Multinomial Naive Bayes classifier uit SK-Learn aan.
  * Train de classifier op de count vectorized train data en bijbehorende targets uit de training set.

In [63]:
clf = MultinomialNB()

clf.fit(Matrix, datasetTrain.target)
#Matrix is de feature vector x in de Naive Bayes
#datasetTrain.Target is de klasse c in de Naive Bayes

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

  * Classificeer de test data met behulp van de getrainde Naive Bayes.
  * Bereken de gemiddelde F1-score (average='macro')

In [76]:
WordFreqTe=np.zeros((len(datasetTest.data),len(vocabTrain)))
WordFreqTe.shape
for idx1, tewords in enumerate(dsTestlower):
    for idx2, vword in enumerate(vocabTrain):
        WordFreqTe[idx1,idx2]=tewords.count(vword)
#print(WordFreqTe)

ypred =clf.predict(WordFreqTe)

In [78]:
errors = 1 - f1_score(datasetTest.target,ypred , average='macro')

In [81]:
print(1-errors)

0.816516111008


In [95]:
from sklearn.metrics import confusion_matrix
confusion_matrix(datasetTest.target,ypred)

array([[293,  99],
       [ 43, 342]])

## 3. TF-IDF vectorizer

In bovenstaande opdracht bestond een feature vector uit het aantal keer dat elk woord voorkomt in een artikel. Alhoewel de methode aardig werkt, wordt hiermee niet meegenomen dat sommige woorden belangrijker zijn om documenten van elkaar te onderscheiden. De term frequency–inverse document frequency neemt dit wel mee.

  * Maak een TF-IDF vectorizer uit SK-Learn aan.
  * Train de vectorizer op de oorspronkelijke nieuwsgroep artikelen (dus niet de gesplitste lijst uit opdracht 2).
  * Transformeer de train en de test data naar TF-IDF vectors.

In [74]:
vectorizer = TfidfVectorizer()

In [75]:
X_train = vectorizer.fit_transform(datasetTrain.data)

In [82]:
X_test = vectorizer.transform(datasetTest.data)

  * Maak een Multinomial Naive Bayes classifier uit SK-Learn aan.
  * Train de classifier op de TF-IDF vectorized train data en bijbehorende targets uit de training set.

In [83]:
clfNB2 = MultinomialNB()

In [84]:
clfNB2.fit(X_train, datasetTrain.target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

  * Classificeer de test data met behulp van de getrainde Naive Bayes.
  * Bereken de gemiddelde F1-score (average='macro'). Vergelijk dit met de score van de count vectorizer.

In [85]:
ypred2 =clfNB2.predict(X_test)

In [86]:
errors = 1 - f1_score(datasetTest.target,ypred2 , average='macro')

In [87]:
print(1-errors)

0.822153722778


  * Kun je de score nog verder verbeteren door stopwoorden te verwijderen, of door de minimale of maximale document frequency aan te passen?

In [89]:
vectorizerSW = TfidfVectorizer(stop_words='english')
X_trainSW = vectorizerSW.fit_transform(datasetTrain.data)
X_testSW = vectorizerSW.transform(datasetTest.data)
clfNB3 = MultinomialNB()
clfNB3.fit(X_train, datasetTrain.target)
ypred3 =clfNB3.predict(X_test)
errors3 = 1 - f1_score(datasetTest.target,ypred3 , average='macro')
print(1-errors3)

0.822153722778


In [96]:
vectorizerSWdfmin = TfidfVectorizer(max_df = 0.9,min_df = 0.02,stop_words='english')
X_trainSWdfmin = vectorizerSWdfmin.fit_transform(datasetTrain.data)
X_testSWdfmin = vectorizerSWdfmin.transform(datasetTest.data)
clfNB4 = MultinomialNB()
clfNB4.fit(X_train, datasetTrain.target)
ypred4=clfNB4.predict(X_test)
errors4 = 1 - f1_score(datasetTest.target,ypred4 , average='macro')
print(1-errors4)

0.822153722778
